In [71]:
# http://www.nutritionresource.com/nutrition-facts/label/cereals-ready-to-eat-general-mills-kix
# Good reference to validate usda data

import numpy as np
import csv
from operator import itemgetter

usda_data = []
for each in range(52):
    usda_filename = 'usda/download (' + str(each) + ').csv'
    with open(usda_filename, encoding = "ISO-8859-1") as f:
        usda_data.append(list(csv.reader(f)))
    
with open('lavasurfer_marked_complete.csv') as f:
    lavasurfer = list(csv.reader(f, delimiter=','))
    

# print(lavasurfer)
lavasurfer_sorted = sorted(lavasurfer, key=lambda k: int(k[0]))
# lavasurfer_sorted = sorted(lavasurfer, key=itemgetter(0)) 
# print(lavasurfer_sorted)
# for row in lavasurfer:
#     for col in lavasurfer:
#         lavasurfer_sorted.append()

col_exclude = ['Water', 'Other', 'Cholesterol']

col_name_mapping = {
    'Energy': 'Calories',
    'Total lipid (fat)': 'Total fat',
    'Carbohydrate by difference': 'Carbs',
    'Fiber total dietary': 'Fiber',
    'Sugars total': 'Sugars',
    'Fatty acids total saturated': 'Saturated Fat',
    'Fatty acids total monounsaturated': 'Monounsaturated Fat',
    'Fatty acids total polyunsaturated': 'Polyunsaturated Fat',
    'Fatty acids total trans': 'Trans Fat'
}
    
usda_dict = []
usda_column_names = []
for file_num in range(len(usda_data)):
    usda_dict.append({})
    for row_num in range(6,len(usda_data[file_num])):
        if (len(usda_data[file_num][row_num]) > 1):
            field_name = usda_data[file_num][row_num][0].replace(',', '')
            
            
            if (field_name in col_name_mapping):
                field_name = col_name_mapping[field_name]
            
            
            if (field_name not in col_exclude):
                if field_name not in usda_column_names:
                    usda_column_names.append(field_name)
                usda_dict[file_num][field_name] = usda_data[file_num][row_num][3]
  
usda_column_names.insert(0, 'Cereal')
usda_column_names.insert(len(usda_column_names), 'Rating')

with open('usda_combined.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(usda_column_names)
    
    for row_num, row in enumerate(usda_dict):
        new_row = []
        new_row.append(lavasurfer_sorted[row_num][1])
        for name in usda_column_names:
            if (name in row):
                new_row.append(row[name])
            else:
                if (name not in ["Cereal", "Rating"]):
                    new_row.append(0)
        new_row.append(float(lavasurfer_sorted[row_num][2]))

        csvwriter.writerow(new_row)
            

In [125]:
from sklearn import model_selection, linear_model, svm, ensemble
import sklearn.model_selection
import numpy

data = numpy.genfromtxt('usda_combined.csv', delimiter=',')[1:,1:]
# print (data[:,len(data[0])-1])
numpy.shape(data)

X_train, X_test, y_train, y_test = model_selection.train_test_split(data[:,0:len(data[0])-1],data[:,len(data[0])-1])

# print(numpy.shape(data))
# print(numpy.shape(X_train))
# print(numpy.shape(X_test))
# print(numpy.shape(y_train))
# print(numpy.shape(y_test))
# print(X_train)

reg = linear_model.ElasticNet()
reg.fit (X_train, y_train)
# print('reg.coef_', reg.coef_)
prediction = reg.predict(X_test)
actual = y_test
# print('predict', reg.predict(X_test))
# print('actual', y_test)

# for each in range(len(prediction)):
#     print (prediction[each], actual[each])
    
# reg = svm.SVR(kernel='linear')
# reg.fit (X_train, y_train)
# # print('reg.coef_', reg.coef_)
# prediction = reg.predict(X_test)
# actual = y_test
# # print('predict', reg.predict(X_test))
# # print('actual', y_test)

# for each in range(len(prediction)):
#     print (prediction[each], actual[each])


reg = ensemble.GradientBoostingRegressor()
reg.fit (X_train, y_train)
# print('reg.coef_', reg.coef_)
prediction = reg.predict(X_test)
actual = y_test
# print('predict', reg.predict(X_test))
# print('actual', y_test)

for each in range(len(prediction)):
    print (prediction[each], actual[each])

5.697967887223738 4.0
4.634671892355202 5.0
5.56610381412578 7.0
6.877915860592966 4.0
5.118023651675163 7.0
7.08167695799643 5.0
6.714962003608904 5.5
5.543834112486982 5.5
4.490036188563867 7.0
5.348879290435356 4.0
6.284110225450684 3.5
5.0586222098077185 2.0
7.07435156866168 7.0
